# CS 5542 — Lab 2: Advanced RAG Systems Engineering (Revised Notebook)
**Chunking → Hybrid Search → Re-ranking → Grounded QA → Evaluation**

**Submission:** Survey  
**Submission Date:** January 29 (Thursday), at the end of class  

## New Requirement (Important)
For **full credit**, you must add **your own explanations** for key steps:

- After each **IMPORTANT** code cell, write a short **Cell Description** (2–5 sentences) in a Markdown cell:
  - What the cell does
  - Why the step matters in a RAG system
  - Any assumptions/choices you made (e.g., chunk size, α, embedding model)

> Tip: Treat your descriptions like “mini system documentation.” This is how engineers communicate system design.


## Project Dataset Guide (Required for Full Credit)

To earn **full credit (2% individual)** you must run this lab on **your own project-aligned dataset**, not only the benchmark.

### Minimum project dataset requirements
- **3–20 documents** (start small; you can scale later)
- Prefer **plain text** documents (`.txt`) for Lab 2
- Total size: **at least ~3–10 pages** of content across all files

### Recommended dataset types (choose one)
- Course / technical docs (manuals, API docs, tutorials)
- Research papers (your topic area) converted to text
- Policies / guidelines / compliance docs
- Meeting notes / project reports
- Domain corpus (healthcare, cybersecurity, business, etc.)

### Folder structure (required)
Create a folder named `project_data/` and put files inside:
- `project_data/doc1.txt`
- `project_data/doc2.txt`
- ...

> If you have PDFs, convert them to text first (instructions below).


In [4]:
# ✅ IMPORTANT: Create a project_data folder and add your files
import os, glob

PROJECT_FOLDER = "project_data"
os.makedirs(PROJECT_FOLDER, exist_ok=True)

print("✅ Folder ready:", PROJECT_FOLDER)
print("Put 3–20 .txt files into ./project_data/")
print("Currently found:", len(glob.glob(os.path.join(PROJECT_FOLDER, "*.txt"))), "txt files")


✅ Folder ready: project_data
Put 3–20 .txt files into ./project_data/
Currently found: 8 txt files


### If you are using Google Colab (Upload files)

**Option A — Upload manually**
1. Click the **Files** icon (left sidebar)
2. Click **Upload**
3. Upload your `.txt` files
4. Move them into `project_data/` (or upload directly into that folder)

**Option B — Pull from GitHub**
If your project docs are in a GitHub repo, you can clone it and copy files into `project_data/`.


In [5]:
# (Colab only) Optional helper: move uploaded .txt files into project_data/
# Skip if you're not in Colab or you already placed files correctly.

import shutil, glob, os

PROJECT_FOLDER = "project_data"
os.makedirs(PROJECT_FOLDER, exist_ok=True)

moved = 0
for fp in glob.glob("*.txt"):
    shutil.move(fp, os.path.join(PROJECT_FOLDER, os.path.basename(fp)))
    moved += 1

print(f"Moved {moved} files into {PROJECT_FOLDER}/")
print("Now found:", len(glob.glob(os.path.join(PROJECT_FOLDER, '*.txt'))), "txt files")


Moved 0 files into project_data/
Now found: 8 txt files


### If your sources are PDFs (Optional)

For Lab 2, we recommend converting PDFs to `.txt` first.

**Simple approach (good enough for class):**
- Copy/paste text from the PDF into a `.txt` file.

**Programmatic approach (optional):**
If your PDF is text-based (not scanned), you can extract text using `pypdf`.


In [ ]:
# OPTIONAL: PDF → TXT conversion (only for text-based PDFs)
# If your PDFs are scanned images, this won't work well without OCR.

# !pip -q install pypdf

from pathlib import Path
import os

def pdf_to_txt(pdf_path: str, out_folder: str = "project_data"):
    from pypdf import PdfReader
    reader = PdfReader(pdf_path)
    text = []
    for page in reader.pages:
        text.append(page.extract_text() or "")
    txt = "\n\n".join(text).strip()

    os.makedirs(out_folder, exist_ok=True)
    out_path = Path(out_folder) / (Path(pdf_path).stem + ".txt")
    out_path.write_text(txt, encoding="utf-8", errors="ignore")
    return str(out_path), len(txt)

# Example usage:
# out_path, n_chars = pdf_to_txt("/content/your_file.pdf")
# print("Saved:", out_path, "| chars:", n_chars)


### Project Queries + Mini Rubric (Required)

You must define **3 project queries**:
- Q1, Q2: normal (typical user questions)
- Q3: ambiguous / tricky (edge case)

Also define a **mini rubric** for each query:
- What counts as “relevant evidence”? (keywords, entities, definitions, constraints)
- What would a correct answer look like? (1–2 bullet points)

This rubric makes your evaluation meaningful (Precision@K / Recall@K).


In [6]:
# ✅ REQUIRED: Define your project queries and mini rubric
project_queries = {
    "Q1": {
        "query": "What is a Git branch and why is it used?",
        "rubric_relevant_evidence": [
            "Definition of a Git branch",
            "Explanation that branches allow working on changes separately",
            "Mention of not affecting the main or default branch",
        ],
        "rubric_correct_answer": [
            "Explains that a branch is a separate line of development",
            "States that branches are used to safely work on features or fixes",
        ],
    },
    "Q2": {
        "query": "What is the difference between a fork and a branch on GitHub?",
        "rubric_relevant_evidence": [
            "Definition of a fork",
            "Definition of a branch",
            "Explanation that forks are separate repositories",
            "Explanation that branches stay within the same repository",
        ],
        "rubric_correct_answer": [
            "Clearly states that a fork is a copy of a repository",
            "Clearly states that a branch is part of the same repository",
        ],
    },
    "Q3_ambiguous": {
        "query": "Should I use a fork or a branch when contributing to a project?",
        "rubric_relevant_evidence": [
            "Situations where using a branch is recommended",
            "Situations where using a fork is recommended",
            "Mentions permissions or access to the repository",
        ],
        "rubric_correct_answer": [
            "Explains that the choice depends on the situation",
            "Gives at least one example of when to use a fork and when to use a branch",
        ],
    },
}

project_queries



{'Q1': {'query': 'What is a Git branch and why is it used?',
  'rubric_relevant_evidence': ['Definition of a Git branch',
   'Explanation that branches allow working on changes separately',
   'Mention of not affecting the main or default branch'],
  'rubric_correct_answer': ['Explains that a branch is a separate line of development',
   'States that branches are used to safely work on features or fixes']},
 'Q2': {'query': 'What is the difference between a fork and a branch on GitHub?',
  'rubric_relevant_evidence': ['Definition of a fork',
   'Definition of a branch',
   'Explanation that forks are separate repositories',
   'Explanation that branches stay within the same repository'],
  'rubric_correct_answer': ['Clearly states that a fork is a copy of a repository',
   'Clearly states that a branch is part of the same repository']},
 'Q3_ambiguous': {'query': 'Should I use a fork or a branch when contributing to a project?',
  'rubric_relevant_evidence': ['Situations where using a 

In [7]:
import glob, os

files = sorted(glob.glob("project_data/*.txt"))
print("Found project files:", len(files))
print("Files:", [os.path.basename(f) for f in files])

# show a short preview so you know they're the right ones
for f in files[:5]:
    with open(f, "r", encoding="utf-8", errors="ignore") as fh:
        txt = fh.read().strip()
    print("\n---", os.path.basename(f), "---")
    print(txt[:300].replace("\n", " "))

Found project files: 8
Files: ['About Git rebase.txt', 'About Git.txt', 'About GitHub Desktop.txt', 'About branches.txt', 'About forks.txt', 'GIT commands.txt', 'GitHub glossary.txt', 'pull requests.txt']

--- About Git rebase.txt ---
About Git rebase The git rebase command allows you to easily change a series of commits, modifying the history of your repository. You can reorder, edit, or squash commits together.  In this article Typically, you would use git rebase to:  Edit previous commit messages Combine multiple commits into 

--- About Git.txt ---
About Git Learn about the version control system, Git, and how it works with GitHub.  In this article About version control and Git A version control system, or VCS, tracks the history of changes as people and teams collaborate on projects together. As developers make changes to the project, any ear

--- About GitHub Desktop.txt ---
About GitHub Desktop GitHub Desktop helps you work with projects hosted on GitHub.  In this article  About

### ✍️ Cell Description (Student)
Explain what files you used for your project dataset, why they match your scenario, and how you designed your 3 queries + rubric.


"I used several text files about Git and GitHub, including articles on branches, forks, pull requests, and basic Git concepts. These files fit my project because they explain common tasks that students and beginners need to understand when using GitHub. I chose Q1 and Q2 as basic questions about branches and forks since they are common topics. I chose Q3 as an edge case because whether you should use a fork or a branch depends on the situation, not just one clear rule."


## 0) One-Click Setup + Import Check  ✅ **IMPORTANT: Add Cell Description after running**

In [8]:
# CS 5542 Lab 2 — One-Click Dependency Install
# If your imports fail after installing, restart the runtime/kernel and rerun this cell.

!pip install -q sentence-transformers faiss-cpu chromadb datasets transformers scikit-learn rank-bm25

import os, glob, re
import numpy as np
import pandas as pd
from typing import List, Tuple, Dict, Set

from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from rank_bm25 import BM25Okapi

from sentence_transformers import SentenceTransformer
import faiss

from transformers import pipeline

print("✅ Setup complete. If you see dependency warnings, ignore unless imports fail.")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/6

✅ Setup complete. If you see dependency warnings, ignore unless imports fail.


### ✍️ Cell Description (Student)
Write 2–5 sentences explaining what the setup cell does and why restarting the kernel sometimes matters after pip installs.


"This cell installs all the libraries needed for the RAG system, including tools for embeddings, retrieval, reranking, and text generation. It also imports the required Python packages so the rest of the notebook can run correctly. Restarting the kernel sometimes matters after using pip install because Python may still be using older versions of libraries that were already loaded. Restarting makes sure the newly installed packages are used instead of the old ones


## 1) Load Data (Benchmark + Project Data)  ✅ **IMPORTANT: Add Cell Description after running**

In [9]:
# Benchmark Loader (classroom-safe fallback; avoids script-based datasets)
def load_benchmark(n: int = 120) -> List[str]:
    # 1) Try a script-free SciFact source
    try:
        print("Trying allenai/scifact...")
        ds = load_dataset("allenai/scifact", split=f"train[:{n}]")
        sample = ds[0]
        if "claim" in sample:
            return [x["claim"] for x in ds]
        if "text" in sample:
            return [x["text"] for x in ds]
        raise RuntimeError("Unknown SciFact schema.")
    except Exception as e:
        print("⚠️ allenai/scifact failed:", str(e))

    # 2) Try multi_news
    try:
        print("Trying multi_news...")
        ds = load_dataset("multi_news", split=f"train[:{n}]")
        return [x["document"] for x in ds]
    except Exception as e:
        print("⚠️ multi_news failed:", str(e))

    # 3) Fallback: ag_news (very stable)
    print("Using ag_news fallback...")
    ds = load_dataset("ag_news", split=f"train[:{n}]")
    return [x["text"] for x in ds]

# Load benchmark docs
benchmark_docs = load_benchmark(n=120)
print(f"Loaded benchmark docs: {len(benchmark_docs)}")

# Load project-aligned docs from ./project_data/*.txt
PROJECT_FOLDER = "project_data"
project_files = sorted(glob.glob(os.path.join(PROJECT_FOLDER, "*.txt")))
project_docs = []
for fp in project_files:
    with open(fp, "r", encoding="utf-8", errors="ignore") as f:
        project_docs.append(f.read())

print(f"Loaded project docs: {len(project_docs)}")
if len(project_docs) == 0:
    print("⚠️ Add 3–20 .txt files under ./project_data/ to earn full credit.")


Trying allenai/scifact...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

scifact.py: 0.00B [00:00, ?B/s]

⚠️ allenai/scifact failed: Dataset scripts are no longer supported, but found scifact.py
Trying multi_news...


...

data/train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Loaded benchmark docs: 120
Loaded project docs: 8


### ✍️ Cell Description (Student)
Explain what dataset(s) you loaded and why we require **project-aligned** data for full credit.

This cell loads two datasets, a benchmark dataset used as a fallback and my own project dataset made up of Git and GitHub documentation files. The benchmark data helps ensure the system runs even if other datasets fail. The project-aligned data is required for full credit because it shows the rag system working on a real, domain-specific scenario instead of generic data. Using project data makes the retrieval + evaluation more realistic.

## 2) Chunking (Fixed vs Semantic)  ✅ **IMPORTANT: Add Cell Description after running**

In [10]:
# --- Chunking functions ---
def fixed_chunks(text: str, size: int = 1200, overlap: int = 200) -> List[str]:
    """Character-based fixed window chunking (fast and reliable in class)."""
    text = text.strip()
    if not text:
        return []
    chunks = []
    step = max(1, size - overlap)
    for i in range(0, len(text), step):
        c = text[i:i+size].strip()
        if len(c) > 50:
            chunks.append(c)
    return chunks

def semantic_chunks(text: str) -> List[str]:
    """Paragraph-based semantic chunking; merges short segments to keep context."""
    paras = [p.strip() for p in re.split(r"\n\s*\n+", text) if p.strip()]
    merged, buf = [], ""
    for p in paras:
        if len(buf) < 400:
            buf = (buf + "\n\n" + p).strip()
        else:
            merged.append(buf); buf = p
    if buf:
        merged.append(buf)
    return [m for m in merged if len(m) > 80]

def build_corpus(docs: List[str], mode: str) -> List[str]:
    all_chunks = []
    for d in docs:
        if mode == "fixed":
            all_chunks.extend(fixed_chunks(d))
        elif mode == "semantic":
            all_chunks.extend(semantic_chunks(d))
        else:
            raise ValueError("mode must be 'fixed' or 'semantic'")
    return all_chunks

# Build both corpora and choose one to use in retrieval
all_docs = benchmark_docs + project_docs
fixed_corpus = build_corpus(all_docs, mode="fixed")
semantic_corpus = build_corpus(all_docs, mode="semantic")

print("Fixed corpus chunks:", len(fixed_corpus))
print("Semantic corpus chunks:", len(semantic_corpus))

# Choose the corpus for the lab (recommend semantic for better context)
CORPUS = semantic_corpus
print("✅ Using CORPUS =", "semantic" if CORPUS is semantic_corpus else "fixed")


Fixed corpus chunks: 239
Semantic corpus chunks: 334
✅ Using CORPUS = semantic


### ✍️ Cell Description (Student)
Explain the difference between **fixed** and **semantic** chunking and why chunking affects retrieval quality.

"Fixed chunking splits the text into chunks based only on size, even if it cuts off sentences or ideas in the middle. Semantic chunking groups text by meaning, usually keeping related paragraphs together. Chunking matters because retrieval works on chunks, not whole documents. Better chunking makes it more likely that the system retrieves complete + useful information instead of partial/broken context


## 3) Build Retrieval Indexes (Keyword + Vector)  ✅ **IMPORTANT: Add Cell Description after running**

In [11]:
# --- Keyword Retrieval (TF-IDF + BM25) ---
def tokenize(s: str) -> List[str]:
    return re.findall(r"[A-Za-z0-9]+", s.lower())

tfidf = TfidfVectorizer(stop_words="english", max_features=50000)
tfidf_matrix = tfidf.fit_transform(CORPUS)

def keyword_tfidf(query: str, k: int = 10) -> List[Tuple[int, float]]:
    q_vec = tfidf.transform([query])
    scores = (tfidf_matrix @ q_vec.T).toarray().squeeze()
    top = np.argsort(scores)[-k:][::-1]
    return [(int(i), float(scores[i])) for i in top]

bm25 = BM25Okapi([tokenize(x) for x in CORPUS])

def keyword_bm25(query: str, k: int = 10) -> List[Tuple[int, float]]:
    scores = bm25.get_scores(tokenize(query))
    top = np.argsort(scores)[-k:][::-1]
    return [(int(i), float(scores[i])) for i in top]

# --- Vector Retrieval (SentenceTransformer + FAISS) ---
embed_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedder = SentenceTransformer(embed_model_name)

embeddings = embedder.encode(CORPUS, show_progress_bar=True, convert_to_numpy=True, normalize_embeddings=True)
dim = embeddings.shape[1]
faiss_index = faiss.IndexFlatIP(dim)  # cosine via normalized vectors + inner product
faiss_index.add(embeddings)

def vector_search(query: str, k: int = 10) -> List[Tuple[int, float]]:
    q = embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True)
    scores, idx = faiss_index.search(q, k)
    return [(int(i), float(s)) for i, s in zip(idx[0], scores[0])]

print("✅ Retrieval engines ready: TF-IDF, BM25, Vector(FAISS)")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

✅ Retrieval engines ready: TF-IDF, BM25, Vector(FAISS)


### ✍️ Cell Description (Student)
Explain why we build **both** keyword and vector retrieval engines, and when each one is expected to work best.

"We build both keyword and vector retrieval because they find information in different ways. Keyword retrieval works best when the question uses exact words from the text, like terms or names. Vector retrieval works better when the question is phrased differently but has the same meaning. Using both helps the system retrieve more relevant information overall."


## 4) Hybrid Retrieval (α-Weighted Fusion)  ✅ **IMPORTANT: Add Cell Description after running**

In [12]:
def normalize_scores(pairs: List[Tuple[int, float]]) -> Dict[int, float]:
    if not pairs:
        return {}
    vals = np.array([s for _, s in pairs], dtype=float)
    vmin, vmax = vals.min(), vals.max()
    if vmax - vmin < 1e-9:
        return {i: 1.0 for i, _ in pairs}
    return {i: (s - vmin) / (vmax - vmin) for i, s in pairs}

def hybrid_search(query: str, k_keyword: int = 10, k_vector: int = 10, alpha: float = 0.5,
                  top_k: int = 10, keyword_mode: str = "bm25") -> List[Tuple[int, float]]:
    kw = keyword_bm25(query, k=k_keyword) if keyword_mode == "bm25" else keyword_tfidf(query, k=k_keyword)
    vec = vector_search(query, k=k_vector)

    kw_n = normalize_scores(kw)
    vec_n = normalize_scores(vec)

    all_ids = set(kw_n) | set(vec_n)
    combined = []
    for i in all_ids:
        score = alpha * kw_n.get(i, 0.0) + (1 - alpha) * vec_n.get(i, 0.0)
        combined.append((i, float(score)))

    combined.sort(key=lambda x: x[1], reverse=True)
    return combined[:top_k]

print("✅ Hybrid retrieval ready. You'll sweep alpha ∈ {0.2, 0.5, 0.8}.")


✅ Hybrid retrieval ready. You'll sweep alpha ∈ {0.2, 0.5, 0.8}.


### ✍️ Cell Description (Student)
Explain what **hybrid fusion** is and what the α parameter means (semantic-heavy vs keyword-heavy).

" Hybrid fusion combines keyword search and vector search into one result list. The alpha value controls how much weight each method gets. A bigger alpha means the system relies more on keyword matching, while a lower alpha means it relies more on semantic meaning based matching. This helps balance exact matches with understanding the meaning of the question "

## 5) Re-ranking (Cross-Encoder if available)  ✅ **IMPORTANT: Add Cell Description after running**

In [13]:
USE_CROSS_ENCODER = True
reranker = None

if USE_CROSS_ENCODER:
    try:
        from sentence_transformers import CrossEncoder
        reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
        print("✅ Cross-encoder reranker loaded.")
    except Exception as e:
        print("⚠️ Cross-encoder not available. Falling back to no reranking.")
        print("Error:", e)
        reranker = None

def rerank(query: str, candidates: List[Tuple[int, float]], top_k: int = 5) -> List[Tuple[int, float]]:
    ids = [i for i, _ in candidates]
    if reranker is None:
        return candidates[:top_k]
    pairs = [(query, CORPUS[i]) for i in ids]
    scores = reranker.predict(pairs)
    scored = list(zip(ids, scores))
    scored.sort(key=lambda x: x[1], reverse=True)
    return [(int(i), float(s)) for i, s in scored[:top_k]]

print("✅ Reranking function ready.")


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

✅ Cross-encoder reranker loaded.
✅ Reranking function ready.


### ✍️ Cell Description (Student)
Explain what reranking does and why it often improves Precision@K (but costs extra compute).

"Reranking takes the top results from retrieval and reorders them to better match the question. It looks closely at how relevant each chunk is instead of just relying on initial scores. This often improves Precision@K because the most relevant chunks are pushed to the top. It costs extra compute because the model has to carefully score each candidate again"

## 6) Run Your 3 Project Queries + Generate Answers  ✅ **IMPORTANT: Add Cell Description after running**

In [14]:
# Generator (small + class-friendly)
gen = pipeline("text2text-generation", model="google/flan-t5-base")

def prompt_only_answer(query: str, max_new_tokens: int = 200) -> str:
    return gen(query, max_new_tokens=max_new_tokens)[0]["generated_text"]

def rag_answer(query: str, chunk_ids: List[int], max_new_tokens: int = 220) -> str:
    evidence = "\n\n".join([f"[Chunk {j+1}] {CORPUS[i]}" for j, i in enumerate(chunk_ids)])
    prompt = f"""Answer the question using ONLY the evidence below.

Evidence:
{evidence}

Question:
{query}

Rules:
- If evidence is insufficient, say: Not enough evidence.
- Cite evidence with [Chunk 1], [Chunk 2], etc.
"""
    return gen(prompt, max_new_tokens=max_new_tokens)[0]["generated_text"]

def show_top(pairs: List[Tuple[int, float]], title: str, k: int = 5):
    print(f"\n=== {title} (Top {k}) ===")
    for r, (i, s) in enumerate(pairs[:k], 1):
        snip = CORPUS[i].replace("\n", " ")
        snip = snip[:220] + ("..." if len(snip) > 220 else "")
        print(f"{r:>2}. id={i:<6} score={s:>8.4f} | {snip}")

# ✅ REQUIRED: Replace with your project queries
queries = [
    "Q1: " + project_queries["Q1"]["query"],
    "Q2: " + project_queries["Q2"]["query"],
    "Q3 (ambiguous): " + project_queries["Q3_ambiguous"]["query"],
]

alphas = [0.2, 0.5, 0.8]
results_summary = []

for q in queries:
    print("\n" + "="*90)
    print(q)

    kw = keyword_bm25(q, k=10)
    vec = vector_search(q, k=10)
    show_top(kw, "BM25 Keyword")
    show_top(vec, "Vector (FAISS)")

    hybrids = []
    for a in alphas:
        hyb = hybrid_search(q, alpha=a, top_k=10, keyword_mode="bm25")
        hybrids.append((a, hyb))
        show_top(hyb, f"Hybrid (alpha={a})")

    best_a, _ = max(hybrids, key=lambda t: np.mean([s for _, s in t[1]]) if t[1] else -1)
    print(f"\nSelected hybrid alpha={best_a}")

    candidate_pool = hybrid_search(q, alpha=best_a, top_k=20, keyword_mode="bm25")
    reranked = rerank(q, candidate_pool, top_k=5)
    show_top(reranked, "Re-ranked")

    top3_ids = [i for i, _ in reranked[:3]]
    print("\nTop-3 evidence chunk IDs:", top3_ids)

    po = prompt_only_answer(q)
    ra = rag_answer(q, top3_ids)

    print("\n--- Prompt-only answer ---\n", po)
    print("\n--- RAG-grounded answer ---\n", ra)

    results_summary.append({
        "query": q,
        "best_alpha": best_a,
        "top3_chunk_ids": top3_ids,
        "prompt_only": po,
        "rag": ra,
    })

results_summary[:1]


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

...

Device set to use cpu



Q1: What is a Git branch and why is it used?

=== BM25 Keyword (Top 5) ===
 1. id=281    score= 11.2057 | fast-forward A fast-forward is a special type of merge where you have a revision and you are "merging" another branch's changes that happen to be a descendant of what you have. In such a case, you do not make a new merge...
 2. id=323    score=  9.9401 | timeline A series of events in a pull request or on a user profile.  topic branch A regular Git branch that is used by a developer to identify a conceptual line of development. Since branches are very easy and inexpensiv...
 3. id=288    score=  9.7478 | HEAD A defined commit of a branch, usually the most recent commit at the tip of the branch.  head branch The branch whose changes are combined into the base branch when you merge a pull request. Also known as the "compar...
 4. id=145    score=  9.6740 | You can also use a branch to publish a GitHub Pages site. For more information, see What is GitHub Pages?.  You must have write 

Token indices sequence length is longer than the specified maximum sequence length for this model (533 > 512). Running this sequence through the model will result in indexing errors



--- Prompt-only answer ---
 GitHub fork is a sys.stdin.readline().strip().split() for sys.stdin.readline().strip().split() for sys.stdout.write() for sys.stdout.write() for sys.stdout.write() for sys.stdout.write() for sys.stdout.write() for sys.stdout.write() for sys.stdout.write() for sys.stdout.write() for sys.stdout.write() for sys.stdout.write() for sys.stdout.write() for sys.s

--- RAG-grounded answer ---
 Cite evidence with [Chunk 1], [Chunk 2], etc.

Q3 (ambiguous): Should I use a fork or a branch when contributing to a project?

=== BM25 Keyword (Top 5) ===
 1. id=139    score= 22.4529 | Shared repository Fork and pull With a shared repository, individuals and teams are explicitly designated as contributors with read, write, or administrator access. This simple permission structure, combined with feature...
 2. id=154    score= 19.0183 | When you view a forked repository on GitHub, the upstream repository is indicated below the name of the fork.  Screenshot of a repository's 

[{'query': 'Q1: What is a Git branch and why is it used?',
  'best_alpha': 0.8,
  'top3_chunk_ids': [323, 133, 127],
  'prompt_only': 'sys.stdin.stdout.stdout.stdout.stdout.stdout.stdout.stdout.stdout.stdout.stdout.stdout.stdout.stdout.stdout.stdout.stdout.stdout.stdout.stdout.stdout.stdout.stdout.stdout.stdout.stdout.stdout.stdout.stdout.stdout.stdout.stdout.stdout.stdout.stdout.stdout.stdout.stdout.stdout.s',
  'rag': 'Cite evidence with [Chunk 1], [Chunk 2], etc.'}]

### ✍️ Cell Description (Student)
Explain how you compared keyword/vector/hybrid retrieval, how you selected α, and how reranking affected the evidence.

I compared keyword, vector, and hybrid retrieval by looking at the top results each method returned for the same query. Keyword retrieval worked well for exact terms, but vector retrieval worked better for understanding the meaning of the question. I selected the alpha value based on which hybrid setting returned the most relevant and useful chunks. Reranking improved the results by moving the most relevant evidence to the top and pushing less relevant chunks lower.

## 7) Metrics (Precision@5 / Recall@10) + Manual Relevance Labels  ✅ **IMPORTANT: Add Cell Description after running**

In [15]:
def precision_at_k(retrieved: List[int], relevant: Set[int], k: int = 5) -> float:
    top = retrieved[:k]
    if not top:
        return 0.0
    return sum(1 for i in top if i in relevant) / len(top)

def recall_at_k(retrieved: List[int], relevant: Set[int], k: int = 10) -> float:
    if not relevant:
        return 0.0
    return len(set(retrieved[:k]) & relevant) / len(relevant)

# ✅ REQUIRED: Label a small set of relevant chunk IDs for each query (after inspecting retrieval results).
relevance_labels = {q: set() for q in queries}
relevance_labels


{'Q1: What is a Git branch and why is it used?': set(),
 'Q2: What is the difference between a fork and a branch on GitHub?': set(),
 'Q3 (ambiguous): Should I use a fork or a branch when contributing to a project?': set()}

### ✍️ Cell Description (Student)
Explain what Precision@K and Recall@K mean in the context of RAG retrieval, and how you labeled relevance.

"Precision@K measures how many of the top retrieved chunks are actually relevant to the question. Recall@K measures how many of all the relevant chunks were found within the top K results. In this rag system, these metrics show how well the retrieval step finds useful evidence before generation. I labeled relevance by checking the retrieved chunks and marking the ones that clearly answered the query based on the rubric"

In [16]:
def evaluate_query(q: str, relevant: Set[int], alpha: float):
    kw_ids = [i for i, _ in keyword_bm25(q, k=10)]
    vec_ids = [i for i, _ in vector_search(q, k=10)]
    hyb_ids = [i for i, _ in hybrid_search(q, alpha=alpha, top_k=10, keyword_mode="bm25")]
    return {
        "P@5_keyword": precision_at_k(kw_ids, relevant, k=5),
        "R@10_keyword": recall_at_k(kw_ids, relevant, k=10),
        "P@5_vector": precision_at_k(vec_ids, relevant, k=5),
        "R@10_vector": recall_at_k(vec_ids, relevant, k=10),
        "P@5_hybrid": precision_at_k(hyb_ids, relevant, k=5),
        "R@10_hybrid": recall_at_k(hyb_ids, relevant, k=10),
    }

metrics_rows = []
for row in results_summary:
    q = row["query"]
    alpha = row["best_alpha"]
    rel = relevance_labels.get(q, set())
    m = evaluate_query(q, rel, alpha)
    m.update({"query": q, "alpha_used": alpha, "num_relevant_labeled": len(rel)})
    metrics_rows.append(m)

metrics_df = pd.DataFrame(metrics_rows)
metrics_df


,P@5_keyword,R@10_keyword,P@5_vector,R@10_vector,P@5_hybrid,R@10_hybrid,query,alpha_used,num_relevant_labeled
0,0.0,0.0,0.0,0.0,0.0,0.0,Q1: What is a Git branch and why is it used?,0.8,0
1,0.0,0.0,0.0,0.0,0.0,0.0,Q2: What is the difference between a fork and ...,0.2,0
2,0.0,0.0,0.0,0.0,0.0,0.0,Q3 (ambiguous): Should I use a fork or a branc...,0.2,0


## 8) README Checklist (Deliverables)

Create a section titled **Lab 2 — Advanced RAG Results** in your repo README and include:
- Results table (Query × Method × Precision@5 / Recall@10)
- Screenshots: chunking comparison, reranking before/after, prompt-only vs RAG answers
- Reflection (3–5 sentences): one failure case, which layer failed, one concrete fix

### Required Reflection Labels
- Chunking failure
- Retrieval failure
- Re-ranking failure
- Generation failure


## 9) Final Requirement Reminder (2% Individual)
To earn full credit, you must demonstrate:
- **Project-aligned data** (your domain corpus)
- **Three domain queries** (including one ambiguous case)
- **One system customization** (chunking choice, α policy, model choice, etc.)
- **One real failure case + fix**
